In [107]:
import numpy as np
import pandas as pd
from xgboost import XGBClassifier
from scipy.stats import linregress
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from xgboost import plot_importance
from matplotlib import pyplot
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import GridSearchCV
sns.set()

In [2]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [3]:
# Reading datasetss
df_stock_train = pd.read_csv("../data/DATA_STOCK_VARS_TRAIN_F.csv")
df_stock_test = pd.read_csv("../data/DATA_STOCK_VARS_TEST_F.csv")

In [8]:
df_stock_train_p = df_stock_train.copy()
df_stock_test_p = df_stock_test.copy()

In [5]:
df_stock_train_p.head(1).T

,0
ID,201808_48117
MES_T0,201808
ID_CLIENTE,48117
FUGA_3M,0
FH_NACIMIENTO,2015-12-04 00:00:00
FH_ALTA,2017-09-04 00:00:00
RGO_RIEGO_BBVA,1
RGO_RIESGO_SIST,8
RGO_SDO_MEDIO_ACTIVO,6
RGO_SDO_MEDIO_PASIVO,9


# Base preprocessing

In [9]:
#Dropping non significant features
df_stock_train_p = df_stock_train_p.drop(['TP_VIVI','TIPO_NEGOCIO'], axis=1)
df_stock_test_p = df_stock_test_p.drop(['TP_VIVI','TIPO_NEGOCIO'], axis=1)

In [10]:
# Tipo persona one hot
df_stock_train_p = pd.concat([df_stock_train_p,pd.get_dummies(df_stock_train_p['TP_PERSONA'], prefix='TP_PERSONA',drop_first=True)],axis=1)
df_stock_test_p = pd.concat([df_stock_test_p,pd.get_dummies(df_stock_test_p['TP_PERSONA'], prefix='TP_PERSONA',drop_first=True)],axis=1)

In [79]:
#Dropping TPPERSONA
df_stock_train_p = df_stock_train_p.drop('TP_PERSONA', axis=1)
df_stock_test_p = df_stock_test_p.drop('TP_PERSONA', axis=1)

In [11]:
# Meses computados segun fecha de nacimiento y alta

# Computando nuevas columnas de meses hasta la fecha de nacimiento y alta.

#Numero de meses hasta la fecha de nacimiento desde año 0 mes 0.
df_stock_train_p['months_NACIMIENTO'] = df_stock_train_p['FH_NACIMIENTO'].apply(lambda x: int(str(x).split('-')[0])*12 + int(str(x).split('-')[1]) if isinstance(x, str) else x)
df_stock_test_p['months_NACIMIENTO'] = df_stock_test_p['FH_NACIMIENTO'].apply(lambda x: int(str(x).split('-')[0])*12 + int(str(x).split('-')[1]) if isinstance(x, str) else x)

#Numero de meses basta fecha de alta desde año 0 mes 0
df_stock_train_p['months_ALTA'] = df_stock_train_p['FH_ALTA'].apply(lambda x: int(str(x).split('-')[0])*12 + int(str(x).split('-')[1]) if isinstance(x, str) else x)
df_stock_test_p['months_ALTA'] = df_stock_test_p['FH_ALTA'].apply(lambda x: int(str(x).split('-')[0])*12 + int(str(x).split('-')[1]) if isinstance(x, str) else x)

#Computando los meses del T0 desde el año 0 mes 0
df_stock_train_p['months_T0'] = df_stock_train_p['MES_T0'].apply(lambda x: int(str(x)[:4])*12 + int(str(x)[4:6]))
df_stock_test_p['months_T0'] = df_stock_test_p['MES_T0'].apply(lambda x: int(str(x)[:4])*12 + int(str(x)[4:6]))

#Computando columnas finales
df_stock_train_p['diff_months_T0_NACIMIENTO'] = df_stock_train_p['months_T0'] - df_stock_train_p['months_NACIMIENTO']
df_stock_test_p['diff_months_T0_NACIMIENTO'] = df_stock_test_p['months_T0'] - df_stock_test_p['months_NACIMIENTO']

df_stock_train_p['diff_months_T0_ALTA'] = df_stock_train_p['months_T0'] - df_stock_train_p['months_ALTA']
df_stock_test_p['diff_months_T0_ALTA'] = df_stock_test_p['months_T0'] - df_stock_test_p['months_ALTA']

df_stock_train_p['diff_months_ALTA_NACIMIENTO'] = df_stock_train_p['months_ALTA'] - df_stock_train_p['months_NACIMIENTO']
df_stock_test_p['diff_months_ALTA_NACIMIENTO'] = df_stock_test_p['months_ALTA'] - df_stock_test_p['months_NACIMIENTO']



In [13]:
df_stock_train_p.head(1).T

,0
ID,201808_48117
MES_T0,201808
ID_CLIENTE,48117
FUGA_3M,0
FH_NACIMIENTO,2015-12-04 00:00:00
FH_ALTA,2017-09-04 00:00:00
RGO_RIEGO_BBVA,1
RGO_RIESGO_SIST,8
RGO_SDO_MEDIO_ACTIVO,6
RGO_SDO_MEDIO_PASIVO,9


# Juntando data con la data computada previamente

In [14]:
#TRANSACCIONES
df_stk_trans_train_wd_agg = pd.read_csv('../data/PROCESS_stk_trans_train_wd_agg.csv')
df_stk_trans_test_wd_agg = pd.read_csv('../data/PROCESS_stk_trans_test_wd_agg.csv')

In [15]:
df_stk_trans_train_wd_agg.head(1).T

,0
ID,201803_1009
RGO_MTO_RECAUDO_sum,78
RGO_MTO_RECAUDO_min,4
RGO_MTO_RECAUDO_max,16
RGO_MTO_RECAUDO_mean,9.75
RGO_MTO_RECAUDO_count,8
RGO_MTO_RECAUDO_std,4.39968
RGO_MTO_TRANSF_INTE_sum,155
RGO_MTO_TRANSF_INTE_min,18
RGO_MTO_TRANSF_INTE_max,20


In [18]:
# Me equivoque xd se deben borrar las columnas de conteo ya que tienen corr 1 (misma info)
count_drop_cols = [count_col for count_col in df_stk_trans_train_wd_agg if '_count' in count_col]
count_drop_cols

['RGO_MTO_RECAUDO_count',
 'RGO_MTO_TRANSF_INTE_count',
 'RGO_MTO_CHEQUE_REC_count',
 'RGO_MTO_TRANSFE_LOCALES_count',
 'RGO_MTO_PAGO_PROVEE_count',
 'RGO_MTO_SUNAT_PYME_count']

In [19]:
count_drop_cols[1:]

['RGO_MTO_TRANSF_INTE_count',
 'RGO_MTO_CHEQUE_REC_count',
 'RGO_MTO_TRANSFE_LOCALES_count',
 'RGO_MTO_PAGO_PROVEE_count',
 'RGO_MTO_SUNAT_PYME_count']

In [20]:
#Droppin and renaming
df_stk_trans_train_wd_agg = df_stk_trans_train_wd_agg.drop(count_drop_cols[1:], axis=1)
df_stk_trans_test_wd_agg = df_stk_trans_test_wd_agg.drop(count_drop_cols[1:], axis=1)

df_stk_trans_train_wd_agg = df_stk_trans_train_wd_agg.rename(columns={'RGO_MTO_RECAUDO_count':'TransCount'})
df_stk_trans_test_wd_agg = df_stk_trans_test_wd_agg.rename(columns={'RGO_MTO_RECAUDO_count':'TransCount'})

In [21]:
df_stk_trans_train_wd_agg.head(1).T

,0
ID,201803_1009
RGO_MTO_RECAUDO_sum,78
RGO_MTO_RECAUDO_min,4
RGO_MTO_RECAUDO_max,16
RGO_MTO_RECAUDO_mean,9.75
TransCount,8
RGO_MTO_RECAUDO_std,4.39968
RGO_MTO_TRANSF_INTE_sum,155
RGO_MTO_TRANSF_INTE_min,18
RGO_MTO_TRANSF_INTE_max,20


In [22]:
#ENDEUDAMIENTO
df_stk_endeu_train_wd_agg = pd.read_csv('../data/PROCESS_stk_endeu_train_wd_agg.csv')
df_stk_endeu_test_wd_agg = pd.read_csv('../data/PROCESS_stk_endeu_test_wd_agg.csv')

In [23]:
df_stk_endeu_train_wd_agg.head(1).T

,0
ID,201803_1003
CD_BANCO_nunique,2
ST_CREDITO_nunique,1
RGO_SDGRANDES_EMP_sum,85
RGO_SDGRANDES_EMP_min,3
RGO_SDGRANDES_EMP_max,13
RGO_SDGRANDES_EMP_mean,7.08333
RGO_SDGRANDES_EMP_count,12
RGO_SDGRANDES_EMP_std,4.42017
RGO_SDMEDIANAS_EMP_sum,98


In [24]:
count_drop_cols = [count_col for count_col in df_stk_endeu_train_wd_agg if '_count' in count_col]
count_drop_cols

['RGO_SDGRANDES_EMP_count',
 'RGO_SDMEDIANAS_EMP_count',
 'RGO_SDPEQUENIAS_EMP_count',
 'RGO_SDMICROEMPRESA_count',
 'RGO_SDINDIRECTO_count',
 'RGO_SDTOTAL_count',
 'RGO_SDDESCLETRAS_count',
 'RGO_SDTARJETA_EMP_count',
 'RGO_SDLEASING_count',
 'RGO_SDCOMEX_count',
 'RGO_SDPREST_EMP_count',
 'RGO_SDFACTORING_count',
 'RGO_SDAVALES_count',
 'RGO_SDCARTFIANZA_count',
 'RGO_SDCREDITO_count',
 'RGO_SDACEPBANCA_count',
 'RGO_SDGTIA_HIPOTEC_count',
 'RGO_SDGTIA_PREFE_count',
 'RGO_SD_NOPREFE_count']

In [25]:
count_drop_cols[1:]

['RGO_SDMEDIANAS_EMP_count',
 'RGO_SDPEQUENIAS_EMP_count',
 'RGO_SDMICROEMPRESA_count',
 'RGO_SDINDIRECTO_count',
 'RGO_SDTOTAL_count',
 'RGO_SDDESCLETRAS_count',
 'RGO_SDTARJETA_EMP_count',
 'RGO_SDLEASING_count',
 'RGO_SDCOMEX_count',
 'RGO_SDPREST_EMP_count',
 'RGO_SDFACTORING_count',
 'RGO_SDAVALES_count',
 'RGO_SDCARTFIANZA_count',
 'RGO_SDCREDITO_count',
 'RGO_SDACEPBANCA_count',
 'RGO_SDGTIA_HIPOTEC_count',
 'RGO_SDGTIA_PREFE_count',
 'RGO_SD_NOPREFE_count']

In [26]:
#Droppin and renaming
df_stk_endeu_train_wd_agg = df_stk_endeu_train_wd_agg.drop(count_drop_cols[1:], axis=1)
df_stk_endeu_test_wd_agg = df_stk_endeu_test_wd_agg.drop(count_drop_cols[1:], axis=1)

df_stk_endeu_train_wd_agg = df_stk_endeu_train_wd_agg.rename(columns={'RGO_SDGRANDES_EMP_count':'DeudasCount'})
df_stk_endeu_test_wd_agg = df_stk_endeu_test_wd_agg.rename(columns={'RGO_SDGRANDES_EMP_count':'DeudasCount'})

In [29]:
# PRODUCTOS
df_stk_prod_train_wd_agg = pd.read_csv('../data/PROCESS_stk_prod_train_wd_agg.csv', skiprows = 1)
df_stk_prod_test_wd_agg = pd.read_csv('../data/PROCESS_stk_prod_test_wd_agg.csv', skiprows = 1)

In [31]:
df_stk_prod_train_wd_agg = df_stk_prod_train_wd_agg.rename(columns={'Unnamed: 0':'ID'})
df_stk_prod_test_wd_agg = df_stk_prod_test_wd_agg.rename(columns={'Unnamed: 0':'ID'})

In [34]:
df_stk_prod_train_wd_agg.head(1).T

,0
ID,201803_100
NU_CTA_AHORRO_min,0
NU_CTA_AHORRO_max,0
RGO_MARGEN_AHORRO_min,2
RGO_MARGEN_AHORRO_max,9
RGO_MARGEN_AHORRO_sum,32
RGO_MARGEN_AHORRO_mean,4
RGO_MARGEN_AHORRO_count,8
RGO_MARGEN_AHORRO_std,2.44949
NU_CTA_CTE_min,1


In [35]:
count_drop_cols = [count_col for count_col in df_stk_prod_train_wd_agg if '_count' in count_col]
count_drop_cols

['RGO_MARGEN_AHORRO_count',
 'RGO_SDO_CTE_count',
 'RGO_MARGEN_CTE_count',
 'RGO_SDO_FMUTUO_count',
 'RGO_SDO_PZO_count',
 'RGO_MARGEN_PZO_count',
 'RGO_SDO_CTS_count',
 'RGO_MARGEN_CTS_count',
 'RGO_SDO_PREST_COM_count',
 'RGO_MARGEN_PREST_COM_count',
 'RGO_SDO_PREST_CONS_count',
 'RGO_MARGEN_PREST_CONS_count',
 'RGO_SDO_PREST_VEHIC_count',
 'RGO_MARGEN_PREST_VEHIC_count',
 'RGO_SDO_PREST_HIP_count',
 'RGO_MARGEN_PREST_HIP_count',
 'RGO_SDO_PAGARE_DESC_count',
 'RGO_MARGEN_PAGARE_DESC_count',
 'RGO_MARGEN_AVAL_count',
 'RGO_SDO_COMEX_count',
 'RGO_SDO_DESC_LETRA_count',
 'RGO_MARGEN_DESC_LETRA_count',
 'RGO_MARGEN_FACTORING_count',
 'RGO_SDO_LSNG_count',
 'RGO_MARGEN_LSNG_count',
 'RGO_SDO_TDC_PNATURAL_count',
 'RGO_SDO_TDC_PJURIDICA_count',
 'RGO_MARGEN_TDC_PJURIDICA_count',
 'RGO_SEGU_VINC_count',
 'RGO_SEGU_NO_VINC_count',
 'RGO_SDO_VALORES_count',
 'RGO_MARGEN_VALORES_count',
 'RGO_MARGEN_COBRANZA_LIB_count',
 'RGO_MARGEN_CONFIRMING_count',
 'RGO_MARGEN_CMPL_count',
 'RGO_MARGEN_C

In [36]:
count_drop_cols[1:]

['RGO_SDO_CTE_count',
 'RGO_MARGEN_CTE_count',
 'RGO_SDO_FMUTUO_count',
 'RGO_SDO_PZO_count',
 'RGO_MARGEN_PZO_count',
 'RGO_SDO_CTS_count',
 'RGO_MARGEN_CTS_count',
 'RGO_SDO_PREST_COM_count',
 'RGO_MARGEN_PREST_COM_count',
 'RGO_SDO_PREST_CONS_count',
 'RGO_MARGEN_PREST_CONS_count',
 'RGO_SDO_PREST_VEHIC_count',
 'RGO_MARGEN_PREST_VEHIC_count',
 'RGO_SDO_PREST_HIP_count',
 'RGO_MARGEN_PREST_HIP_count',
 'RGO_SDO_PAGARE_DESC_count',
 'RGO_MARGEN_PAGARE_DESC_count',
 'RGO_MARGEN_AVAL_count',
 'RGO_SDO_COMEX_count',
 'RGO_SDO_DESC_LETRA_count',
 'RGO_MARGEN_DESC_LETRA_count',
 'RGO_MARGEN_FACTORING_count',
 'RGO_SDO_LSNG_count',
 'RGO_MARGEN_LSNG_count',
 'RGO_SDO_TDC_PNATURAL_count',
 'RGO_SDO_TDC_PJURIDICA_count',
 'RGO_MARGEN_TDC_PJURIDICA_count',
 'RGO_SEGU_VINC_count',
 'RGO_SEGU_NO_VINC_count',
 'RGO_SDO_VALORES_count',
 'RGO_MARGEN_VALORES_count',
 'RGO_MARGEN_COBRANZA_LIB_count',
 'RGO_MARGEN_CONFIRMING_count',
 'RGO_MARGEN_CMPL_count',
 'RGO_MARGEN_CARTERA_count',
 'RGO_PASIVO_

In [37]:
#Droppin and renaming
df_stk_prod_train_wd_agg = df_stk_prod_train_wd_agg.drop(count_drop_cols[1:], axis=1)
df_stk_prod_test_wd_agg = df_stk_prod_test_wd_agg.drop(count_drop_cols[1:], axis=1)

df_stk_prod_train_wd_agg = df_stk_prod_train_wd_agg.rename(columns={'RGO_MARGEN_AHORRO_count':'ProductCount'})
df_stk_prod_test_wd_agg = df_stk_prod_test_wd_agg.rename(columns={'RGO_MARGEN_AHORRO_count':'ProductCount'})

In [38]:
df_stk_prod_train_wd_agg.head(1).T

,0
ID,201803_100
NU_CTA_AHORRO_min,0
NU_CTA_AHORRO_max,0
RGO_MARGEN_AHORRO_min,2
RGO_MARGEN_AHORRO_max,9
RGO_MARGEN_AHORRO_sum,32
RGO_MARGEN_AHORRO_mean,4
ProductCount,8
RGO_MARGEN_AHORRO_std,2.44949
NU_CTA_CTE_min,1


In [46]:
#Cotizaciones
df_stk_coti_train_wd_agg= pd.read_csv('../data/PROCESS_stk_coti_train_wd_agg.csv')# , skiprows = 1)
df_stk_coti_test_wd_agg = pd.read_csv('../data/PROCESS_stk_coti_test_wd_agg.csv' , skiprows = 1)

In [48]:
df_stk_coti_train_wd_agg.head()

,ID,ESTADO_unique,ESTADO_count,ESTADO_DESEMBOLSO_COTIZA_count,ESTADO_DESEMBOLSO_COTIZA_unique,DIVISA_unique,DIVISA_count,RGO_IMPORTE_min,RGO_IMPORTE_max,RGO_IMPORTE_sum,...,RGO_IMPORTE_DESEMBOLSADO_MN_min,RGO_IMPORTE_DESEMBOLSADO_MN_max,RGO_IMPORTE_DESEMBOLSADO_MN_sum,RGO_IMPORTE_DESEMBOLSADO_MN_mean,RGO_IMPORTE_DESEMBOLSADO_MN_std,RGO_TASA_DESEMBOLSO_min,RGO_TASA_DESEMBOLSO_max,RGO_TASA_DESEMBOLSO_sum,RGO_TASA_DESEMBOLSO_mean,RGO_TASA_DESEMBOLSO_std
0,201803_11238,['TARIFARIO'],2,2,['DESEMBOLSADA'],['PEN'],2,16,16,32,...,17,17,34,17.0,0.000000,20,20,40,20.0,0.000000
1,201803_11239,['APROBADA'],1,1,['PENDIENTE'],['PEN'],1,10,10,10,...,2,2,2,2.0,NaN,8,8,8,8.0,NaN
2,201803_11275,['RECUPERADA'],1,1,['nan'],['PEN'],1,2,2,2,...,11,11,11,11.0,NaN,4,4,4,4.0,NaN
3,201803_11303,['APROBADA'],2,2,['PENDIENTE'],['USD'],2,19,19,38,...,7,8,15,7.5,0.707107,8,9,17,8.5,0.707107
4,201803_11813,['APROBADA'],1,1,['PENDIENTE'],['USD'],1,1,1,1,...,9,9,9,9.0,NaN,10,10,10,10.0,NaN


In [49]:
#No estoy seguro d que paso con el train xd
df_stk_coti_test_wd_agg = df_stk_coti_test_wd_agg.rename(columns={'Unnamed: 0':'ID'})

In [50]:
df_stk_coti_test_wd_agg.head()

,ID,ESTADO_unique,ESTADO_count,ESTADO_DESEMBOLSO_COTIZA_count,ESTADO_DESEMBOLSO_COTIZA_unique,DIVISA_unique,DIVISA_count,RGO_IMPORTE_min,RGO_IMPORTE_max,RGO_IMPORTE_sum,...,RGO_IMPORTE_DESEMBOLSADO_MN_min,RGO_IMPORTE_DESEMBOLSADO_MN_max,RGO_IMPORTE_DESEMBOLSADO_MN_sum,RGO_IMPORTE_DESEMBOLSADO_MN_mean,RGO_IMPORTE_DESEMBOLSADO_MN_std,RGO_TASA_DESEMBOLSO_min,RGO_TASA_DESEMBOLSO_max,RGO_TASA_DESEMBOLSO_sum,RGO_TASA_DESEMBOLSO_mean,RGO_TASA_DESEMBOLSO_std
0,201812_18473,['APROBADA'],2,2,['PENDIENTE'],['PEN'],2,8.0,11.0,19.0,...,3.0,14.0,17.0,8.500000,7.778175,2.0,15.0,17.0,8.500000,9.192388
1,201812_19207,['APROBADA' 'MODIFICADA'],3,3,['DESEMBOLSADA' 'nan' 'PENDIENTE'],['PEN'],3,1.0,6.0,8.0,...,3.0,14.0,25.0,8.333333,5.507571,6.0,16.0,28.0,9.333333,5.773503
2,201812_19357,['TARIFARIO'],1,1,['PENDIENTE'],['USD'],1,1.0,1.0,1.0,...,16.0,16.0,16.0,16.000000,NaN,14.0,14.0,14.0,14.000000,NaN
3,201812_19588,['APROBADA'],1,1,['PENDIENTE'],['PEN'],1,20.0,20.0,20.0,...,20.0,20.0,20.0,20.000000,NaN,18.0,18.0,18.0,18.000000,NaN
4,201812_19851,['TARIFARIO'],1,1,['DESEMBOLSADA'],['USD'],1,20.0,20.0,20.0,...,4.0,4.0,4.0,4.000000,NaN,2.0,2.0,2.0,2.000000,NaN


In [52]:
#Dropearemos unique columns solo por falta de tiempo de momento.
unique_drop_cols = [unique_col for unique_col in df_stk_coti_train_wd_agg if '_unique' in unique_col]
unique_drop_cols

['ESTADO_unique',
 'ESTADO_DESEMBOLSO_COTIZA_unique',
 'DIVISA_unique',
 'UNIDAD_PLAZO_unique',
 'GARANTIA_unique']

In [53]:
# Count
count_drop_cols = [count_col for count_col in df_stk_coti_train_wd_agg if '_count' in count_col]
count_drop_cols

['ESTADO_count',
 'ESTADO_DESEMBOLSO_COTIZA_count',
 'DIVISA_count',
 'RGO_IMPORTE_count',
 'UNIDAD_PLAZO_count',
 'GARANTIA_count']

In [54]:
df_stk_coti_train_wd_agg = df_stk_coti_train_wd_agg.drop(unique_drop_cols, axis=1)
df_stk_coti_test_wd_agg = df_stk_coti_test_wd_agg.drop(unique_drop_cols, axis=1)

df_stk_coti_train_wd_agg = df_stk_coti_train_wd_agg.drop(count_drop_cols[1:], axis=1)
df_stk_coti_test_wd_agg = df_stk_coti_test_wd_agg.drop(count_drop_cols[1:], axis=1)

df_stk_coti_train_wd_agg = df_stk_coti_train_wd_agg.rename(columns={'ESTADO_count':'CotiCount'})
df_stk_coti_test_wd_agg = df_stk_coti_test_wd_agg.rename(columns={'ESTADO_count':'CotiCount'})

In [58]:
#SOLICITUDES
df_stk_soli_train_wd_agg = pd.read_csv('../data/PROCESS_stk_soli_train_wd_agg.csv', skiprows = 1)
df_stk_soli_test_wd_agg = pd.read_csv('../data/PROCESS_stk_soli_test_wd_agg.csv', skiprows = 1)

In [59]:
df_stk_soli_test_wd_agg.head()

,Unnamed: 0,Moneda_unique,Moneda_count,RGO_ImporteS_min,RGO_ImporteS_max,RGO_ImporteS_sum,RGO_ImporteS_mean,RGO_IMPORTE_std,Producto_Solicitado_nunique
0,201812_18473,['SOLES'],1,9,9,9,9.0,NaN,1
1,201812_19207,['SOLES'],1,2,2,2,2.0,NaN,1
2,201812_19760,['DÓLARES'],1,1,1,1,1.0,NaN,1
3,201812_19851,['DÓLARES'],2,13,15,28,14.0,1.414214,2
4,201812_19892,['SOLES'],1,16,16,16,16.0,NaN,1


In [60]:
df_stk_soli_train_wd_agg = df_stk_soli_train_wd_agg.rename(columns={'Unnamed: 0':'ID'})
df_stk_soli_test_wd_agg = df_stk_soli_test_wd_agg.rename(columns={'Unnamed: 0':'ID'})

In [61]:
#u know
df_stk_soli_train_wd_agg = df_stk_soli_train_wd_agg.drop('Moneda_unique', axis=1)
df_stk_soli_test_wd_agg = df_stk_soli_test_wd_agg.drop('Moneda_unique', axis=1)

df_stk_soli_train_wd_agg = df_stk_soli_train_wd_agg.rename(columns={'Moneda_count':'SoliCount'})
df_stk_soli_test_wd_agg = df_stk_soli_test_wd_agg.rename(columns={'Moneda_count':'SoliCount'})

# El mega merge y ajustes finales

In [80]:
#Merging train
df_stock_train_f = df_stock_train_p.merge(df_stk_trans_train_wd_agg,on=['ID'], how = 'left')
df_stock_train_f = df_stock_train_f.merge(df_stk_endeu_train_wd_agg,on=['ID'], how = 'left')
df_stock_train_f = df_stock_train_f.merge(df_stk_prod_train_wd_agg,on=['ID'], how = 'left')
df_stock_train_f = df_stock_train_f.merge(df_stk_coti_train_wd_agg,on=['ID'], how = 'left')
df_stock_train_f = df_stock_train_f.merge(df_stk_soli_train_wd_agg,on=['ID'], how = 'left')


#Merging test
df_stock_test_f = df_stock_test_p.merge(df_stk_trans_test_wd_agg,on=['ID'], how = 'left')
df_stock_test_f = df_stock_test_f.merge(df_stk_endeu_test_wd_agg,on=['ID'], how = 'left')
df_stock_test_f = df_stock_test_f.merge(df_stk_prod_test_wd_agg,on=['ID'], how = 'left')
df_stock_test_f = df_stock_test_f.merge(df_stk_coti_test_wd_agg,on=['ID'], how = 'left')
df_stock_test_f = df_stock_test_f.merge(df_stk_soli_test_wd_agg,on=['ID'], how = 'left')

In [81]:
cols_to_drop_original_ds = ['MES_T0','ID_CLIENTE','FH_NACIMIENTO','FH_ALTA','months_NACIMIENTO',
                            'months_ALTA','months_T0']

In [82]:
df_stock_train_f = df_stock_train_f.drop(cols_to_drop_original_ds, axis=1)
df_stock_test_f = df_stock_test_f.drop(cols_to_drop_original_ds, axis=1)

In [83]:
df_stock_train_f.shape

(73661, 414)

In [84]:
df_stock_test_f.shape

(26338, 414)

# Model i guess xd

In [85]:
final_cols = list(set(df_stock_train_f.columns)-set(['ID','FUGA_3M']))

In [86]:
X = df_stock_train_f[final_cols]
y = df_stock_train_f['FUGA_3M']

In [87]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42, stratify = y)

In [88]:
y_train.value_counts(normalize = True)

0    0.866277
1    0.133723
Name: FUGA_3M, dtype: float64

In [89]:
y_test.value_counts(normalize = True)

0    0.866287
1    0.133713
Name: FUGA_3M, dtype: float64

In [98]:
#idk
xgb = XGBClassifier(learning_rate =0.05,
                 n_estimators=600,
                 max_depth=5,
                 min_child_weight=1,
                 gamma=0,
                 subsample=0.8,
                 colsample_bytree=0.8,
                 objective= 'binary:logistic',
                 nthread=4,
                 scale_pos_weight=1,
                 seed=42)

In [99]:
xgb.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=0.8, gamma=0,
       learning_rate=0.05, max_delta_step=0, max_depth=5,
       min_child_weight=1, missing=None, n_estimators=600, n_jobs=1,
       nthread=4, objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=42, silent=None,
       subsample=0.8, verbosity=1)

In [101]:
#Getting results
y_pred = xgb.predict_proba(X_test)

In [102]:
y_pred_final = y_pred[:,1]

In [103]:
y_pred_final

array([0.03482463, 0.02183263, 0.05052224, ..., 0.01659894, 0.30824876,
       0.1613056 ], dtype=float32)

In [104]:
#resultinn
roc_auc_score(y_test, y_pred_final)

0.827925801541655

In [97]:
xgb.get_booster().get_score(importance_type= 'gain')

{'RGO_RIESGO_SIST': 126.68287398060869,
 'RGO_SEGU_NO_VINC_mean': 8.852923663977936,
 'RGO_SDO_MEDIO_PASIVO': 53.134813522832175,
 'RGO_SDO_VALORES_std': 12.983561420194249,
 'NU_CTA_TDD_min': 9.176406621818181,
 'RGO_MARGEN_DESC_LETRA_max': 4.94098235722,
 'RGO_SDO_PAGARE_DESC_sum': 7.33737064499107,
 'RGO_SDO_DESC_LETRA_std': 6.2175102907345305,
 'RGO_SDCOMEX_mean': 5.105143843443036,
 'RGO_SDO_TDC_PJURIDICA_sum': 4.46569886092308,
 'RGO_SDO_PZO_std': 5.307892954365519,
 'RGO_MARGEN_AHORRO_mean': 4.74857732901739,
 'TP_PERSONA_J': 51.70007901617648,
 'RGO_SDAVALES_mean': 10.824170159583339,
 'CD_SBS_NORMAL': 16.112434516794526,
 'RGO_SDCARTFIANZA_std': 5.5100433241631555,
 'RGO_SDO_MEDIO_ACTIVO': 11.951577528249997,
 'NU_CTA_NOMINA_max': 20.478987436666653,
 'RGO_SDMICROEMPRESA_max': 12.614123147315789,
 'RGO_MARGEN_COBRANZA_LIB_std': 5.147139290466321,
 'RGO_MARGEN_PREST_VEHIC_std': 5.3034309833142865,
 'CD_SBS_CPP': 6.616879805163635,
 'NU_CTA_TDD_max': 18.275988669714284,
 'RGO_SE

## Inter variables


In [105]:
df_stock_train_f.head(1).T

,0
ID,201808_48117
FUGA_3M,0
RGO_RIEGO_BBVA,1
RGO_RIESGO_SIST,8
RGO_SDO_MEDIO_ACTIVO,6
RGO_SDO_MEDIO_PASIVO,9
RGO_MARGEN_OPER_MES,1
RGO_MARGEN_OPER_ACUM,1
RGO_FACTURA_ANUAL,7
TO_EMPLEADOS,2


In [108]:
df_stock_train_f['RGO_BBVA_SIST'] = df_stock_train_f['RGO_RIESGO_SIST']*df_stock_train_f['RGO_RIEGO_BBVA']
df_stock_test_f['RGO_BBVA_SIST'] = df_stock_test_f['RGO_RIESGO_SIST']*df_stock_test_f['RGO_RIEGO_BBVA']

df_stock_train_f['RGO_ACTIVOXPASIVO'] = df_stock_train_f['RGO_SDO_MEDIO_ACTIVO']*df_stock_train_f['RGO_SDO_MEDIO_PASIVO']
df_stock_test_f['RGO_ACTIVOXPASIVO'] = df_stock_test_f['RGO_SDO_MEDIO_ACTIVO']*df_stock_test_f['RGO_SDO_MEDIO_PASIVO']

df_stock_train_f['RGO_ACTIVOENPASIVO'] = df_stock_train_f['RGO_SDO_MEDIO_ACTIVO']/df_stock_train_f['RGO_SDO_MEDIO_PASIVO']
df_stock_test_f['RGO_ACTIVOENPASIVO'] = df_stock_test_f['RGO_SDO_MEDIO_ACTIVO']/df_stock_test_f['RGO_SDO_MEDIO_PASIVO']

df_stock_train_f['log_TO_EMPLEADOS'] = np.sqrt(df_stock_train_f.TO_EMPLEADOS)
df_stock_test_f['log_TO_EMPLEADOS'] = np.sqrt(df_stock_test_f.TO_EMPLEADOS)

df_stock_train_f['ratio_ALTA_NAC'] = df_stock_train_f['diff_months_T0_ALTA']/df_stock_train_f['diff_months_T0_NACIMIENTO']
df_stock_test_f['ratio_ALTA_NAC'] = df_stock_test_f['diff_months_T0_ALTA']/df_stock_test_f['diff_months_T0_NACIMIENTO']



In [117]:
X = df_stock_train_f[final_cols]
y = df_stock_train_f['FUGA_3M']

In [118]:
# A ver un treini
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 23, stratify = y)

xgb = XGBClassifier(learning_rate =0.05,
                 n_estimators=600,
                 max_depth=5,
                 min_child_weight=1,
                 gamma=0,
                 subsample=0.8,
                 colsample_bytree=0.8,
                 objective= 'binary:logistic',
                 nthread=4,
                 scale_pos_weight=1,
                 seed=42)

xgb.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=0.8, gamma=0,
       learning_rate=0.05, max_delta_step=0, max_depth=5,
       min_child_weight=1, missing=None, n_estimators=600, n_jobs=1,
       nthread=4, objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=42, silent=None,
       subsample=0.8, verbosity=1)

In [119]:
y_pred = xgb.predict_proba(X_test)

In [120]:
y_pred_final = y_pred[:,1]
roc_auc_score(y_test, y_pred_final)

0.8273243047498897

In [ ]:
#eee

In [110]:
#GridSearch
param_test1 = {
 'max_depth':range(3,5,7),
 'min_child_weight':range(1,5,3)
}
gsearch1 = GridSearchCV(estimator = XGBClassifier(learning_rate =0.05,
                 n_estimators=600,
                 max_depth=5,
                 min_child_weight=1,
                 gamma=0,
                 subsample=0.8,
                 colsample_bytree=0.8,
                 objective= 'binary:logistic',
                 nthread=4,
                 scale_pos_weight=1,
                 seed=42), 
 param_grid = param_test1, scoring='roc_auc',n_jobs=4,iid=False, cv=5)

gsearch1.fit(X,y)

/Users/dascencios/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
/Users/dascencios/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
/Users/dascencios/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
/Users/dascencios/anaconda3/lib/python3.7/site-packages/sklearn/ext

AttributeError: 'GridSearchCV' object has no attribute 'grid_scores_'

In [112]:
gsearch1.cv_results_

/Users/dascencios/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/Users/dascencios/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/Users/dascencios/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split2_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/Users/dascencio

{'mean_fit_time': array([ 591.48967581,  588.91487827,  675.8618001 ,  907.522855  ,
         880.83118978,  864.90942221, 1214.88345513, 1193.33763876,
        1175.0975738 , 1587.35307627, 1524.29676065, 1375.09138474]),
 'std_fit_time': array([ 49.95386465,  44.31458384,  62.2819774 , 100.55423395,
         79.16297812,  78.31898232, 103.77321684, 102.41681579,
        122.3655675 , 130.2498254 , 127.52729874, 217.75379491]),
 'mean_score_time': array([0.6231936 , 0.54367948, 0.56476889, 0.68855906, 0.65296874,
        0.65247836, 0.82447762, 0.83185349, 0.81770134, 1.0657155 ,
        0.98291149, 0.71810284]),
 'std_score_time': array([0.05886874, 0.0391975 , 0.03782332, 0.01586687, 0.01530202,
        0.01600661, 0.04247929, 0.0361013 , 0.01379969, 0.03972866,
        0.06787966, 0.20385024]),
 'param_max_depth': masked_array(data=[3, 3, 3, 5, 5, 5, 7, 7, 7, 9, 9, 9],
              mask=[False, False, False, False, False, False, False, False,
                    False, False, Fals

In [113]:
gsearch1.best_score_

0.4853005090971769

In [114]:
gsearch1.best_params_

{'max_depth': 3, 'min_child_weight': 3}

TypeError: score() missing 2 required positional arguments: 'X' and 'y'